# Order Latency Data

To obtain more realistic backtesting results, accounting for latencies is crucial. Therefore, it's important to collect both feed data and order data with timestamps to measure your order latency. The best approach is to gather your own order latencies. You can collect order latency based on your live trading or by regularly submitting orders at a price that cannot be filled and then canceling them for recording purposes. However, if you don't have access to them or want to establish a target, you will need to artificially generate order latency. You can model this latency based on factors such as feed latency, trade volume, and the number of events. In this guide, we will demonstrate a simple method to generate order latency from feed latency using a multiplier and offset for adjustment.

First, loads the feed data.

In [1]:
import numpy as np

data = np.load('gate/btc_usdt_20240829.npz')['data']
data

array([(3758096385, 1724889599996000000, 1724889600016258443, 59008.1,    0., 0, 0, 0.),
       (3758096385, 1724889599996000000, 1724889600016258443, 58999.7,    0., 0, 0, 0.),
       (3758096385, 1724889599996000000, 1724889600016258443, 58999.6, 2119., 0, 0, 0.),
       ...,
       (3758096385, 1724975999982000000, 1724975999997128781, 59302.9,    0., 0, 0, 0.),
       (3758096385, 1724975999982000000, 1724975999997128781, 59302.1,    0., 0, 0, 0.),
       (3489660929, 1724975999982000000, 1724975999997128781, 59343.3,    0., 0, 0, 0.)],
      dtype=[('ev', '<u8'), ('exch_ts', '<i8'), ('local_ts', '<i8'), ('px', '<f8'), ('qty', '<f8'), ('order_id', '<u8'), ('ival', '<i8'), ('fval', '<f8')])

For easy manipulation, converts it into a DataFrame.

In [2]:
import polars as pl

df = pl.DataFrame(data)
df

ev,exch_ts,local_ts,px,qty,order_id,ival,fval
u64,i64,i64,f64,f64,u64,i64,f64
3758096385,1724889599996000000,1724889600016258443,59008.1,0.0,0,0,0.0
3758096385,1724889599996000000,1724889600016258443,58999.7,0.0,0,0,0.0
3758096385,1724889599996000000,1724889600016258443,58999.6,2119.0,0,0,0.0
3758096385,1724889599996000000,1724889600016258443,58992.0,0.0,0,0,0.0
3489660929,1724889599996000000,1724889600016258443,59021.8,6434.0,0,0,0.0
…,…,…,…,…,…,…,…
3758096385,1724975999864000000,1724975999877134412,59278.5,0.0,0,0,0.0
3489660929,1724975999882000000,1724975999897689282,59343.4,0.0,0,0,0.0
3758096385,1724975999982000000,1724975999997128781,59302.9,0.0,0,0,0.0


Selects only the events that have both a valid exchange timestamp and a valid local timestamp to get feed latency.

In [3]:
from hftbacktest import EXCH_EVENT, LOCAL_EVENT

df = df.filter((pl.col('ev') & EXCH_EVENT == EXCH_EVENT) & (pl.col('ev') & LOCAL_EVENT == LOCAL_EVENT))

Reduces the number of rows by resampling to approximately 1-second intervals.

In [4]:
df = df.with_columns(
    pl.col('local_ts').alias('ts')
).group_by_dynamic(
    'ts', every='1000000000i'
).agg(
    pl.col('exch_ts').last(),
    pl.col('local_ts').last()
).drop('ts')

df

exch_ts,local_ts
i64,i64
1724889600934000000,1724889600956409830
1724889601976000000,1724889601996377936
1724889602981000000,1724889602995114123
1724889603982000000,1724889603996405400
1724889604980000000,1724889604996802657
…,…
1724975995975000000,1724975995996533148
1724975996940000000,1724975996956210624
1724975997942000000,1724975997956611480


Converts back to the structured NumPy array.

In [5]:
data = df.to_numpy(structured=True)
data

array([(1724889600934000000, 1724889600956409830),
       (1724889601976000000, 1724889601996377936),
       (1724889602981000000, 1724889602995114123), ...,
       (1724975997942000000, 1724975997956611480),
       (1724975998984000000, 1724975998999567244),
       (1724975999982000000, 1724975999997128781)],
      dtype=[('exch_ts', '<i8'), ('local_ts', '<i8')])

Generates order latency. Order latency consists of two components: the latency until the order request reaches the exchange's matching engine and the latency until the response arrives backto the localy. Order latency is not the same as feed latency and does not need to be proportional to feed latency. However, for simplicity, we model order latency to be proportional to feed latency using a multiplier and offset.

In [6]:
mul_entry = 4
offset_entry = 0

mul_resp = 3
offset_resp = 0

order_latency = np.zeros(len(data), dtype=[('req_ts', 'i8'), ('exch_ts', 'i8'), ('resp_ts', 'i8'), ('_padding', 'i8')])
for i, (exch_ts, local_ts) in enumerate(data):
    feed_latency = local_ts - exch_ts
    order_entry_latency = mul_entry * feed_latency + offset_entry
    order_resp_latency = mul_resp * feed_latency + offset_resp

    req_ts = local_ts
    order_exch_ts = req_ts + order_entry_latency
    resp_ts = order_exch_ts + order_resp_latency
    
    order_latency[i] = (req_ts, order_exch_ts, resp_ts, 0)
    
order_latency

array([(1724889600956409830, 1724889601046049150, 1724889601113278640, 0),
       (1724889601996377936, 1724889602077889680, 1724889602139023488, 0),
       (1724889602995114123, 1724889603051570615, 1724889603093912984, 0),
       ...,
       (1724975997956611480, 1724975998015057400, 1724975998058891840, 0),
       (1724975998999567244, 1724975999061836220, 1724975999108537952, 0),
       (1724975999997128781, 1724976000057643905, 1724976000103030248, 0)],
      dtype=[('req_ts', '<i8'), ('exch_ts', '<i8'), ('resp_ts', '<i8'), ('_padding', '<i8')])

In [7]:
df_order_latency = pl.DataFrame(order_latency)
df_order_latency

req_ts,exch_ts,resp_ts,_padding
i64,i64,i64,i64
1724889600956409830,1724889601046049150,1724889601113278640,0
1724889601996377936,1724889602077889680,1724889602139023488,0
1724889602995114123,1724889603051570615,1724889603093912984,0
1724889603996405400,1724889604054027000,1724889604097243200,0
1724889604996802657,1724889605064013285,1724889605114421256,0
…,…,…,…
1724975995996533148,1724975996082665740,1724975996147265184,0
1724975996956210624,1724975997021053120,1724975997069684992,0
1724975997956611480,1724975998015057400,1724975998058891840,0


Checks if latency has invalid negative values.

In [8]:
order_entry_latency = df_order_latency['exch_ts'] - df_order_latency['req_ts']
order_resp_latency = df_order_latency['resp_ts'] - df_order_latency['exch_ts']

In [9]:
(order_entry_latency <= 0).sum()

0

In [10]:
(order_resp_latency <= 0).sum()

0

Here, we wrap the entire process into a method with `njit` for increased speed.

In [11]:
import numpy as np
from numba import njit
import polars as pl
from hftbacktest import LOCAL_EVENT, EXCH_EVENT

@njit
def generate_order_latency_nb(data, order_latency, mul_entry, offset_entry, mul_resp, offset_resp):
    for i in range(len(data)):
        exch_ts = data[i].exch_ts
        local_ts = data[i].local_ts
        feed_latency = local_ts - exch_ts
        order_entry_latency = mul_entry * feed_latency + offset_entry
        order_resp_latency = mul_resp * feed_latency + offset_resp

        req_ts = local_ts
        order_exch_ts = req_ts + order_entry_latency
        resp_ts = order_exch_ts + order_resp_latency

        order_latency[i].req_ts = req_ts
        order_latency[i].exch_ts = order_exch_ts
        order_latency[i].resp_ts = resp_ts

def generate_order_latency(feed_file, output_file = None, mul_entry = 1, offset_entry = 0, mul_resp = 1, offset_resp = 0):
    data = np.load(feed_file)['data']
    df = pl.DataFrame(data)
    
    df = df.filter(
        (pl.col('ev') & EXCH_EVENT == EXCH_EVENT) & (pl.col('ev') & LOCAL_EVENT == LOCAL_EVENT)
    ).with_columns(
        pl.col('local_ts').alias('ts')
    ).group_by_dynamic(
        'ts', every='1000000000i'
    ).agg(
        pl.col('exch_ts').last(),
        pl.col('local_ts').last()
    ).drop('ts')
    
    data = df.to_numpy(structured=True)

    order_latency = np.zeros(len(data), dtype=[('req_ts', 'i8'), ('exch_ts', 'i8'), ('resp_ts', 'i8'), ('_padding', 'i8')])
    generate_order_latency_nb(data, order_latency, mul_entry, offset_entry, mul_resp, offset_resp)

    if output_file is not None:
        np.savez_compressed(output_file, data=order_latency)

    return order_latency

In [12]:
order_latency = generate_order_latency('gate/btc_usdt_20240829.npz', output_file='gate/feed_latency_20240829.npz', mul_entry=4, mul_resp=3)

KeyboardInterrupt: 